# Front-end for the Flask app
This is the front-end for the basic-flask example.  It is a simple Jupyter notebook showing how to use the requests library to connect to the Flask app.
 
Before running this notebook, it will be required to run the Flask app.  This can be done by running the following command in a terminal:
 
```bash
python3 basic-flask.py
```


## GET all the customers in the database
While this isn't always the most practical approach (to get everything), we are looking for simple here, so we're going to get all the customers.  This first cell will demonstrate how to make a generic GET request to the Flask app.

In [38]:
import requests
from flask import jsonify
import json

# Set our endpoint
url = "http://localhost:5000/customers"

# We'll reserve the payload for POST/PUT methods generally
payload = {}

# We don't have any specific headers, but this can be used for authentication if required
headers = {}

# Make the request
response = requests.request("GET", url, headers=headers, data=payload)

# The response object contains a lot of information about the response, including the status code
print(f"Respose Status: {response.status_code}")
# We can also get the response body as text, json, html or other.  In our case, we know it's json so let's just ask for that
customers = response.json()

for customer in customers:
    print(customer)

Respose Status: 200
[2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5]
[3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3]
[4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4]
[5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4]
[6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5]
[7, 'Astrid', 'Gruber', None, 'Rotenturmstraße 4, 1010 Innere Stadt', 'Vienne', None, 'Austria', '1010', '+43 01 5134505', None, 'astrid.gruber@apple.at', 5]
[8, 'Daan', 'Peeters', None, 'Grétrystraat 63', 'Brussels', None, 'Belgi

Perfect!  We've gotten back a list of customers, but they are pretty difficult to deal with.  I mean we have to parse the JSON, and then we have to deal with the fact that the data is in a list.  It would be much easier if we could get the data into Customer objects which can then be passed around a little easier. Let's do that.

In [ ]:
class Customer():
    
    def __init__(self, id, first_name, last_name, company, address, city, state, country, postal_code, phone, fax, email, support_rep_id ) :
        self.id = id
        self.first_name = first_name
        self.last_name = last_name
        self.company = company
        self.address = address
        self.city = city
        self.state = state
        self.country = country
        self.postal_code = postal_code
        self.phone = phone
        self.fax = fax
        self.email = email
        self.support_rep_id = support_rep_id
    
    def to_json():
        customer_dict = {"id": self.id,
                         "first_name": self.first_name,
                         "last_name": self.last_name,
                         "company": self.company,
                         "address": self.address,
                         "city": self.city,
                         "state": self.state,
                         "country": self.country,
                         "postal_code": self.postal_code,
                         "phone": self.phone,
                         "fax": self.fax,
                         "email": self.email,
                         "support_rep_id": self.support_rep_id}
        return customer_dict.jsonify()

In [32]:
# Let's store our customers in a dictionary.  The lookup will be by last name
customer_directory = {}
for item in customers:
    customer = Customer(item[0], item[1], item[2], item[3], item[4], item[5], item[6],item[7], item[8], item[9], item[10], item[11], item[12])
    customer_directory[customer.last_name] = customer

# Now we can lookup a specific customer using there last name
customer_goyer = customer_directory['Goyer']
print( customer_goyer)
print(customer_goyer.first_name)

Tim


## GET a specific customer
Our API also allows us to get a specific customer by passing in the customer id.  Let's try that out.  Remember the format of the request is
`/customers/<customer_id>` where `<customer_id>` is the id of the customer we want to get.

Again we could go through the same thing and turn this into an object of type `Customer`.  For brevity we are going to leave this alone.

In [36]:
# Set our endpoint
url = "http://localhost:5000/customers"
url = url + "/" + str(customer_goyer.id)

# Again no specific payload or headers
payload = {}
headers = {}

# Make the request
response = requests.request("GET", url, headers=headers, data=payload)

# The response object contains a lot of information about the response, including the status code
print(f"Response Status: {response.status_code}")

customer = response.json()
print(customer)

Respose Status: 200
[19, 'Tim', 'Goyer', 'Apple Inc.', '1 Infinite Loop', 'Cupertino', 'CA', 'USA', '95014', '+1 (408) 996-1010', '+1 (408) 996-1011', 'tgoyer@apple.com', 3]


## POST a new customer
Now that we've gotten a customer, let's add a new one.  We can do this by making a POST request to the Flask app.  The format of the request is `/customers` and we need to pass in a JSON object with the customer data.  Let's try that out.

A couple of things to note here.  Even though we are using a dictionary, which looks like valid JSON, we are going to have the json library ensure it's write by dumping the string to json.

Second, we have to tell the Flask app that we are sending JSON data.  We do this by setting the `Content-Type` header to `application/json`.

In [40]:
# Set our endpoint
url = "http://localhost:5000/customers"

# Now let's build our payload
new_customer = json.dumps({
    "FirstName": "John",
    "LastName": "Doe",
    "Company": "Acme, Inc.",
    "Address": "123 Main St.",
    "City": "Anytown",
    "State": "CA",
    "Country": "USA",
    "PostalCode": "12345",
    "Phone": "555-555-5555",
    "Fax": "555-555-5555"
    })
# We need to set the content type header
headers = {'Content-Type': "application/json"}

# Make the request
response = requests.request("POST", url, headers=headers, data=new_customer)

# The response object contains a lot of information about the response, including the status code
print(f"Response Status: {response.status_code}")

Response Status: 200
None


Now, we expect to see our new record at the end of the list if we query for the customers again, so let's try.

In [41]:
# Set our endpoint
url = "http://localhost:5000/customers"

# We'll reserve the payload for POST/PUT methods generally
payload = {}

# We don't have any specific headers, but this can be used for authentication if required
headers = {}

# Make the request
response = requests.request("GET", url, headers=headers, data=payload)

# The response object contains a lot of information about the response, including the status code
print(f"Response Status: {response.status_code}")
# We can also get the response body as text, json, html or other.  In our case, we know it's json so let's just ask for that
customers = response.json()

# This says to give us the last item in the list
print(customers[-1])

Response Status: 200
[61, 'John', 'Doe', 'Acme, Inc.', '123 Main St.', 'Anytown', 'CA', 'USA', '12345', '555-555-5555', '555-555-5555', '', None]


## PUT an update to a customer
If you look closely at the last customer we just entered.  We forgot to put in their email address, oops!  Let's go ahead and update the customer.

In order to update the customer, we need to make a PUT request to the Flask app.  The format of the request is `/customers/<customer_id>` where `<customer_id>` is the id of the customer we want to update.  We also need to pass in a JSON object with the customer data.  Let's try that out.

In [58]:
# Customer id, this we are getting from the first field in the last item in the list
customer_id = customers[-1][0]
print(customer_id)

# Set our endpoint
url = "http://localhost:5000/customers"

# Now let's build our payload, the fields we want to update
customer_update = json.dumps({
    "CustomerId": customer_id,
    "Email": "john.doe@example.com"
     })
# Again We need to set the content type header
headers = {'Content-Type': "application/json"}

# Make the request - notice we are using the PUT method
response= requests.put(url, headers=headers, data=customer_update)

print(response.status_code)
# In this case, the result is the updated customer record
print(response.json())

61
200
[61, 'John', 'Doe', 'Acme, Inc.', '123 Main St.', 'Anytown', 'AR', 'USA', '12345', '123-555-1234', '555-555-5555', 'john.doe@example.com', None]


## DELETE a customer
Now that we have updated the customer, let's go ahead and delete them.  We can do this by making a DELETE request to the Flask app.  The format of the request is `/customers/<customer_id>` where `<customer_id>` is the id of the customer we want to delete.  Let's try that out.

In [59]:
print(f"Deleting customer with id: {customer_id}")

# Set our endpoint
url = f"http://localhost:5000/customers/{customer_id}"

# Our payload is empty, just like with the get request
payload = {}

# And since we aren't sending any payload we don't have any headers
headers = {}

# Make the request - again we could use the verb as a parameter, but this is more explicit
response= requests.delete(url, headers=headers, data=customer_update)

print(response.status_code)


Deleting customer with id: 61
200
